In [1]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "focus_group"

In [1]:
from fcgb.chatbots.chatbot import BaseChatBot
from typing import Dict, Any, List, Annotated
from pydantic import BaseModel
from langgraph.constants import Send
from operator import add
from fcgb.types.research import SimpleTaskModel
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage
#from fcgb.chatbots.selfconv import SimpleTaskDistributionChatBot

In [ ]:
class SimpleTaskDistributionChatBot(BaseChatBot):
    """
    A chatbot that distributes tasks for a single LLM answer.
    """

    def __init__(
            self, 
            llm,
            initial_messages_spec,
            internal_messages_spec,
            memory,
            global_inputs={},
            init_values={},
            prompt_manager_spec = {}
        ):

        super().__init__(
            llm=llm,
            initial_messages_spec=initial_messages_spec,
            internal_messages_spec=internal_messages_spec,
            memory=memory,
            init_values=init_values,
            prompt_manager_spec=prompt_manager_spec,
            global_inputs=global_inputs,
            compile=False
            )

        self.compile_graph()

    def _set_state_class(self):

        task_list_model = self.internal_messages_spec['task_list']['answer_format']
        task_answer_model = self.internal_messages_spec['task']['answer_format']
        answer_model = self.internal_messages_spec['answer']['answer_format']

        class VerificationState(BaseModel):
            parent_thread_id: str 
            template_inputs: Dict[str, str]
            task_list: task_list_model | None # type: ignore
            simple_task_response: Annotated[List[task_answer_model], add] # type: ignore
            verified_answer: answer_model | None # type: ignore

        self.state_class = VerificationState

    def _set_tasks_gen_func(self):
        """
        Set the function that generates the verification tasks.
        """
        
        def tasks_gen_func(state: self.state_class) -> Dict: # type: ignore
            
            task_list = self._invoke_internal_msg(name='task_list', template_inputs=state.template_inputs)

            return {'task_list': task_list}
        
        return tasks_gen_func
        
    def _set_tasks_router_func(self):
        """
        Set the function that distributes tasks along nodes.
        """
        
        def tasks_router_func(state: self.state_class) -> SimpleTaskModel: # type: ignore

            return [Send('task_node',
                        {
                        'template_inputs': state.template_inputs,
                        'task': task,
                        'simple_task_response': None
                        }) for task in state.task_list.prompts]
        
        return tasks_router_func
    
    def _set_task_node_func(self):
        """
        Set the function that solve the tasks.
        """
        
        def task_node_func(state: SimpleTaskModel) -> Dict:

            system = self.internal_prompts['task']['prompt'].format(**state['template_inputs'], **self.global_inputs)
            prompt = state['task'].format(**state['template_inputs'], **self.global_inputs)

            messages = [SystemMessage(content=system), HumanMessage(content=prompt)]
            task_response = self.llm.with_structured_output(self.internal_prompts['task']['answer_format']).invoke(messages)

            return {'simple_task_response': [task_response]}
        
        return task_node_func
    
    def _set_output_node_func(self):

        def output_node_func(state: self.state_class) -> Dict: # type: ignore

            task_response = '\n-----\n'.join([f'Job {i+1}:\n'+resp.recommendations for i, resp in enumerate(state.simple_task_response)])

            output = self._invoke_internal_msg(name='answer', template_inputs=state.template_inputs | {'task_response': task_response})

            return {'verified_answer': output}
        
        return output_node_func
    
    def _compile_graph(self):

        task_gen_func = self._set_tasks_gen_func()
        tasks_router_func = self._set_tasks_router_func()
        task_node_func = self._set_task_node_func()
        output_node_func = self._set_output_node_func()

        workflow = StateGraph(self.state_class)
        workflow.add_node('task_gen', task_gen_func)
        workflow.add_node('task_node', task_node_func)
        workflow.add_node('output_node', output_node_func)

        workflow.add_edge(START, 'task_gen')
        workflow.add_conditional_edges('task_gen', tasks_router_func, ['task_node'])
        workflow.add_edge('task_node', 'output_node')
        workflow.add_edge('output_node', END)

        self.graph = workflow.compile(
            checkpointer=self.memory,
        )

    def run(self, template_inputs: Dict, thread_id: str, parent_thread_id: str):

        self.init_thread(thread_id=thread_id, template_inputs=template_inputs)

        config = self._get_config(thread_id)

        return self.graph.invoke({'parent_thread_id': parent_thread_id, 'template_inputs': template_inputs}, config=config)

## fake llm test

In [2]:
self_research_inputs = {
    'template_inputs': {
        'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
        'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.',
    },
    'ssc_thread_id': 'test_2/ssc1',
    'parent_thread_id': 'test_2'
}

In [3]:
from fcgb.chatbots.specbots import SelfConvForStrategySpecBot, StrategizedSelfResearchSpecBot
from fcgb.fake_models import FakeLLM
from langgraph.checkpoint.memory import MemorySaver

fake_llm = FakeLLM()

memory = MemorySaver()
convbot = SelfConvForStrategySpecBot(llm=fake_llm, memory=memory)
strategybot = StrategizedSelfResearchSpecBot(llm=fake_llm, self_conv_bot=convbot, memory=memory)

In [4]:
research_output = strategybot.run(**self_research_inputs)
research_output

{'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.'},
 'ssc_thread_id': 'test_2/ssc1',
 'parent_thread_id': 'test_2',
 'ssc_summary': SimpleAnswerModel(answer='Fake string srxrs'),
 'strategies': StrategyTaskModel(strategies=[{'strategy_description': 'Fake string vywvv', 'paraphrased_task': 'Fake string drije', 'paraphrased_context': 'Fake string pnuhd'}, {'strategy_description': 'Fake string bdwql', 'paraphrased_task': 'Fake string aaouo', 'paraphrased_context': 'Fake string fijkg'}, {'strategy_description': 'Fake string upjna', 'paraphrased_task': 'Fake string ibxnn', 'paraphrased_context': 'Fake string jwxsu'}, {'strategy_description': 'Fake string belfk', 'paraphrased_task': 'Fake string faehz', 'paraphrased_context': 'Fake string dqrea'}]),
 'sc_thread

In [5]:
verification_inputs ={
    'template_inputs': self_research_inputs['template_inputs'] | {'answer': research_output['ssc_summary'].answer},
    'thread_id': self_research_inputs['ssc_thread_id'] + '/ver',
    'parent_thread_id': self_research_inputs['ssc_thread_id']
}

In [6]:
from fcgb.cfg.chat_inputs_spec import ResearchVerificationConfig

verificationbot = SimpleTaskDistributionChatBot(
    llm=fake_llm,
    initial_messages_spec=ResearchVerificationConfig.initial_messages_spec,
    internal_messages_spec=ResearchVerificationConfig.internal_messages_spec,
    memory=memory,
    global_inputs=ResearchVerificationConfig.global_inputs,
    init_values=ResearchVerificationConfig.init_values,
    prompt_manager_spec=ResearchVerificationConfig.prompt_manager_spec
)

In [7]:
verification_output = verificationbot.run(**verification_inputs)
verification_output

{'parent_thread_id': 'test_2/ssc1',
 'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. It is dedicated to potential users of LLMs who want to learn how to efficiently use LLMs.',
  'answer': 'Fake string srxrs'},
 'task_list': PromptTemplatesListModel(analysis='Fake string ukgjw', prompts=['Fake string klyze', 'Fake string jlaxq', 'Fake string janjk']),
 'simple_task_response': [SingleVerificationModel(analysis='Fake string dvqxz', recommendations='Fake string cvqeq'),
  SingleVerificationModel(analysis='Fake string apynn', recommendations='Fake string nuqma'),
  SingleVerificationModel(analysis='Fake string xeudp', recommendations='Fake string suvmg')],
 'verified_answer': SimpleAnswerModel(answer='Fake string vzojy')}

## real llm test

In [3]:
from fcgb.cfg.precompiled import get_checkpointer, get_llm

memory = get_checkpointer(checkpointer_mode='mongodb', mode='test')
llm = get_llm(llm_model='google')

In [53]:
from fcgb.chatbots.specbots import SelfConvForStrategySpecBot, StrategizedSelfResearchSpecBot
from fcgb.cfg.chat_inputs_spec import ResearchVerificationConfig

convbot = SelfConvForStrategySpecBot(llm=llm, memory=memory)
strategybot = StrategizedSelfResearchSpecBot(llm=llm, self_conv_bot=convbot, memory=memory)
verificationbot = SimpleTaskDistributionChatBot(
    llm=llm,
    initial_messages_spec=ResearchVerificationConfig.initial_messages_spec,
    internal_messages_spec=ResearchVerificationConfig.internal_messages_spec,
    memory=memory,
    global_inputs=ResearchVerificationConfig.global_inputs,
    init_values=ResearchVerificationConfig.init_values,
    prompt_manager_spec=ResearchVerificationConfig.prompt_manager_spec
)

In [ ]:
"""from fcgb.cfg.precompiled import get_db_client

db = get_db_client(db_engine='mongodb', mode='test')

db['checkpoints_writes'].delete_many({'thread_id': 'partial/research/cot/ssc1'})
db['checkpoints'].delete_many({'thread_id': 'partial/research/cot/ssc1'})

strategybot.get_state('partial/research/cot/ssc1')"""

In [54]:
self_research_inputs = {
    'template_inputs': {
        'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
        'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. List should be very comprehensive, be sure to include all known variants of CoT prompting techniques. Be aware to not include fine-tuning methods, as they are not related to prompting.',
    },
    'ssc_thread_id': 'partial/research/cot/ssc7',
    'parent_thread_id': 'partial/research/cot'
}

In [10]:
research_output = strategybot.run(**self_research_inputs)
research_output

{'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. List should be very comprehensive, be sure to include all known variants of CoT prompting techniques.'},
 'ssc_thread_id': 'partial/research/cot/ssc7',
 'parent_thread_id': 'partial/research/cot',
 'ssc_summary': SimpleAnswerModel(answer='The variants of Chain-of-Thought (CoT) prompting techniques are:\n\n*   Standard CoT\n*   Zero-shot CoT\n*   Self-Consistency\n*   Finetuned CoT\n*   Least-to-Most prompting\n*   Automatic Prompt Optimization (APO)\n*   CoT with Verification\n*   Graph CoT\n*   Tree-of-Thoughts (ToT)\n*   CoT combined with Knowledge Retrieval\n*   Distillation for CoT\n*   Complexity-Based Prompting\n*   Task-Specific CoT\n*   Self-Refinement\n*   CoT with External Verification\n*   Constraint-Based CoT\n*   Graph-of-Thoughts (GoT)\n*   Program-of-Thoughts (PoT)\n*   Chain-of-H

In [6]:
research_output = strategybot.get_state(self_research_inputs['ssc_thread_id'])
research_output

{'template_inputs': {'task': 'List all variants of Chain-of-Thought (CoT) prompting techniques.',
  'context': 'CoT prompting techniques list is necessary to plan a chapter about CoT prompting. List should be very comprehensive, be sure to include all known variants of CoT prompting techniques.'},
 'ssc_thread_id': 'partial/research/cot/ssc7',
 'parent_thread_id': 'partial/research/cot',
 'ssc_summary': SimpleAnswerModel(answer='The variants of Chain-of-Thought (CoT) prompting techniques are:\n\n*   Standard CoT\n*   Zero-shot CoT\n*   Self-Consistency\n*   Finetuned CoT\n*   Least-to-Most prompting\n*   Automatic Prompt Optimization (APO)\n*   CoT with Verification\n*   Graph CoT\n*   Tree-of-Thoughts (ToT)\n*   CoT combined with Knowledge Retrieval\n*   Distillation for CoT\n*   Complexity-Based Prompting\n*   Task-Specific CoT\n*   Self-Refinement\n*   CoT with External Verification\n*   Constraint-Based CoT\n*   Graph-of-Thoughts (GoT)\n*   Program-of-Thoughts (PoT)\n*   Chain-of-H

In [16]:
for line in research_output['ssc_summary'].answer.split('\n'):
    print(line)

The variants of Chain-of-Thought (CoT) prompting techniques are:

*   Standard CoT
*   Zero-shot CoT
*   Self-Consistency
*   Finetuned CoT
*   Least-to-Most prompting
*   Automatic Prompt Optimization (APO)
*   CoT with Verification
*   Graph CoT
*   Tree-of-Thoughts (ToT)
*   CoT combined with Knowledge Retrieval
*   Distillation for CoT
*   Complexity-Based Prompting
*   Task-Specific CoT
*   Self-Refinement
*   CoT with External Verification
*   Constraint-Based CoT
*   Graph-of-Thoughts (GoT)
*   Program-of-Thoughts (PoT)
*   Chain-of-Hindsight (CoH)
*   CoT-Guided ToT/GoT
*   CoT for Branch Selection in ToT
*   CoT for Node Generation in GoT


In [55]:
verification_inputs ={
    'template_inputs': self_research_inputs['template_inputs'] | {'answer': research_output['ssc_summary'].answer},
    'thread_id': self_research_inputs['ssc_thread_id'] + '/ver12',
    'parent_thread_id': self_research_inputs['ssc_thread_id']
}

In [ ]:
verification_output = verificationbot.run(**verification_inputs)
verification_output

In [ ]:
# task delegation analysis
for line in verification_output['task_list'].analysis.split('\n'):
    print(line)

1. **Task Decomposition:** The task is to list all variants of Chain-of-Thought (CoT) prompting techniques. The answer format is a list.

2. **Contextual Restrictions and Information:**
    - The list should be comprehensive.
    - The list should not include fine-tuning methods.
    - The list should only include prompting techniques.

3. **Answer Analysis and Doubts:**
    - The answer provides a list of CoT prompting techniques. However, some entries might be fine-tuning methods (which should be excluded based on the context). Also, the comprehensiveness of the list is questionable and needs verification. There might be missing CoT variants. Some listed items may be duplicates or very similar techniques described with different names. The correctness of each listed item needs to be verified to ensure it's a valid CoT prompting technique and not a misclassified or non-existent method.
    - Some of the items might be too general, e.g. 'CoT combined with Knowledge Retrieval' is a very

In [ ]:
# prompts list
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['task_list'].prompts:
    for line in sc_summ.split('\n'):
        print(line)
    print(sep)

Job: Verify that each item in the list is a valid Chain-of-Thought (CoT) *prompting* technique and not a fine-tuning method or something else.
Doubts: Some items might be misclassified or not directly related to CoT prompting as defined in the context.
Additional Info: The context specifies that fine-tuning methods should be excluded. Focus on techniques that guide the LLM during inference, not during training.
Analysis methodology: 1. For each item in the list, research its definition and usage.
2. Determine if the item is a prompting technique or a fine-tuning method.
3. If it's a prompting technique, verify that it aligns with the Chain-of-Thought approach (i.e., involves generating intermediate reasoning steps).
4. If it's not a valid CoT prompting technique, flag it for removal or modification.

----------------------------------------

Job: Check the comprehensiveness of the list. Are there any significant and well-known CoT prompting techniques missing?
Doubts: The list might no

In [ ]:
# analysis from all tasks
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['simple_task_response']:
    for line in sc_summ.analysis.split('\n'):
        print(line)
    print(sep)

I will analyze each item in the provided list to determine if it is a valid Chain-of-Thought (CoT) prompting technique, ensuring it is not a fine-tuning method or an unrelated technique. I will consult external resources to confirm the definition and usage of each item. If an item is misclassified or not directly related to CoT prompting, I will flag it for removal or modification.

----------------------------------------

I will conduct a search for recent surveys and research papers on Chain-of-Thought prompting to identify CoT variants mentioned in these sources that are not present in the provided list and add any missing variants to the list.

----------------------------------------

I have compared all pairs of items in the list and identified potential duplicates or very similar techniques. For example, 'Graph CoT' and 'Graph-of-Thoughts (GoT)' seem very similar, and 'CoT-Guided ToT/GoT', 'CoT for Branch Selection in ToT', and 'CoT for Node Generation in GoT' also appear close

In [ ]:
# separate recommendations from all tasks
sep = f'\n{'-' * 40}\n'
for sc_summ in verification_output['simple_task_response']:
    for line in sc_summ.recommendations.split('\n'):
        print(line)
    print(sep)

No changes are needed. The list appears to be comprehensive and accurate. All listed items are valid variants or applications of Chain-of-Thought (CoT) prompting techniques. Therefore, no modifications are required at this time. The answer is correct and comprehensive

----------------------------------------

The list seems relatively comprehensive, but it could benefit from the inclusion of the following CoT variants: - Multimodal CoT: This variant incorporates multiple modalities, such as text and images, into the CoT process. - Step-Back Prompting: This involves prompting the model to take a step back and consider the bigger picture before answering a question. - Generated Knowledge Prompting: This technique encourages the model to generate relevant knowledge before answering a question. Adding these techniques would make the list more complete and up-to-date. Also consider merging similar techniques, such as Graph CoT and Graph-of-Thoughts (GoT), and explicitly mentioning their re

In [32]:
for line in verification_output['verified_answer'].answer.split('\n'):
    print(line)

The variants of Chain-of-Thought (CoT) prompting techniques are:

*   Standard CoT
*   Zero-shot CoT
*   Self-Consistency
*   Finetuned CoT
*   Least-to-Most prompting
*   Automatic Prompt Optimization (APO)
*   CoT with Verification
*   Graph CoT
*   Tree-of-Thoughts (ToT)
*   Distillation for CoT
*   Complexity-Based Prompting
*   Task-Specific CoT
*   Self-Refinement
*   CoT with External Verification
*   Constraint-Based CoT
*   Graph-of-Thoughts (GoT)
*   Program-of-Thoughts (PoT)
*   Chain-of-Hindsight (CoH)
*   CoT-Guided ToT/GoT
*   CoT for Branch Selection in ToT
*   CoT for Node Generation in GoT
*   Multimodal CoT
*   Step-Back Prompting
*   Generated Knowledge Prompting
*   Knowledge Retrieval Augmented CoT
